## Build a Simple Application with LCEL

In this quickstart, we'll show how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of 
* Using language models
* Using PromptTemplate and OutputParsers
* Using LangChain Expression Language(LCEL) to chain components together
* Debugging and tracing your application using LangSmith
* Deploying your application using LangServe

In [5]:
###  Open AI API key and Open Source models -- llama3, Gemma2, mistral -- Groq

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [13]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7d6a1c8a6a20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7d6a1c8a9fd0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [17]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello! How are you?")
]

result = model.invoke(messages)
result

AIMessage(content='Bonjour ! Comment vas-tu ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 22, 'total_tokens': 33, 'completion_time': 0.02, 'prompt_time': 0.001337569, 'queue_time': 0.25009641, 'total_time': 0.021337569}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b9e774db-a162-42f4-8932-5c9e512284fd-0', usage_metadata={'input_tokens': 22, 'output_tokens': 11, 'total_tokens': 33})

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser() #  display the content message only 
parser.invoke(result)

'Bonjour ! Comment allez-vous ? \n'

### Using LCEL - Chain the components

In [16]:
chain = model | parser
chain.invoke(messages)

"Bonjour ! Comment allez-vous ? \n\n\nLet me know if you'd like to translate anything else!\n"

### Prompt Templates

instead of using messages we can use this for efficiency

In [18]:
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user", "{text}")
    ]
)

In [21]:
prompt.invoke({"language": "french", "text" : "hello"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into french:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hello', additional_kwargs={}, response_metadata={})])

In [22]:
chain = prompt | model | parser
chain.invoke({"language": "korean", "text": "Hello"})

'안녕하세요 (Annyeonghaseyo) \n'